In [7]:
import ee

ee.Authenticate(auth_mode='notebook')
ee.Initialize()

In [8]:
# Import other packages used in the tutorial
%matplotlib inline
import geemap
import numpy as np
import random, time
import matplotlib.pyplot as plt
from scipy.stats import norm, chi2

from pprint import pprint  # for pretty printing

In [9]:
# Truncate a 1-D array to dec decimal places
def trunc(values, dec =3):
  return np.trunc(values*10**dec)/(10**dec)

In [10]:
# Display an image in a one percent linear stretch
def display_ls(image, map, name, centered=False):
  bns = image.bandNames().length().getInfo()
  if bns==3:
    image = image.rename('B1','B2','B3')
    pb_99=['B1_p99','B2_p99','B3_p99']
    pb_1=['B1_p1','B2_p1', 'B3_p1']
    img=ee.Image.rgb(image.select('B1'), image.select('B2'),image.select('B3'))
  else:
    image=image.rename('B1')
    pb_99 = ['B1_p99']
    pb_1=['B1_p1']
    img=image.select('B1')
  percentiles= image.reduceRegion(ee.Reducer.percentile([1,99]), maxPixels=1e11)
  mx = percentiles.values(pb_99)
  if centered:
    mn = ee.Array(mx).multiply(-1).toList()
  else:
    mn = percentiles.values(pb_1)
  map.addLayer(img, {'min':mn, 'max':mx},name)